### Uses processing.py's code in a notebook for interactivity. Also attempts some contour length calculations. Doesn't yet convert to physical units.

This code generates a lot of `.txt` and `.png` files in the imgproc/ directory. They're gitignored so don't show up in GitHub, but you can view them locally.

# Setup

In [10]:
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from ipywidgets import interact, fixed
import cv2

from imgproc_helpers import *

OUTPUT_DIR = '../../out'
NRRD1_PATH = '../../ExampleData/BCP_Dataset_2month_T1w.nrrd'
NRRD2_PATH = '../../ExampleData/IBIS_Dataset_12month_T1w.nrrd'
NRRD3_PATH = '../../ExampleData/IBIS_Dataset_NotAligned_6month_T1w.nrrd'
"""This is an intentionally misaligned image."""
NIFTI_PATH = '../../ExampleData/MicroBiome_1month_T1w.nii.gz'

reader = sitk.ImageFileReader()
reader.SetFileName(NIFTI_PATH)
image = reader.Execute()
print(f'Dimensions: {image.GetSize()[0]}, {image.GetSize()[1]}, {image.GetSize()[2]}')

Dimensions: 244, 292, 198


# Interactive version of `rotate_and_get_contour`

`rotate_and_get_contour` is already defined in `imgproc_helpers.py` but is redefined here to include `plt.show()` to display with `ipywidgets`. I also comment out the smooth slice since it causes errors to appear in the output

To be clear, this is not "production-quality" code. It's just for interactivity for testing purposes.

In [11]:
def rotate_and_get_contour_testing(img: sitk.Image, theta_x: int, theta_y: int, theta_z: int, slice_z: int) -> sitk.Image:
    """Same as the function of the same name in imgproc_helpers.py but needs to be redefined here to include `plt.show()`."""

    euler_3d_transform = sitk.Euler3DTransform()
    # NOTE: This center is possibly incorrect.
    euler_3d_transform.SetCenter(img.TransformContinuousIndexToPhysicalPoint([((dimension - 1) / 2.0) for dimension in img.GetSize()]))
    euler_3d_transform.SetRotation(degrees_to_radians(theta_x), degrees_to_radians(theta_y), degrees_to_radians(theta_z))
    rotated_image = sitk.Resample(img, euler_3d_transform)
    rotated_image_slice = rotated_image[:,:,slice_z]

    # The cast is necessary, otherwise get sitk::ERROR: Pixel type: 16-bit signed integer is not supported in 2D
    # However, this does throw some weird errors
    # smooth_slice = sitk.GradientAnisotropicDiffusionImageFilter().Execute(sitk.Cast(rotated_image_slice, sitk.sitkFloat64))

    otsu = sitk.OtsuThresholdImageFilter().Execute(rotated_image_slice)

    hole_filling = sitk.BinaryGrindPeakImageFilter().Execute(otsu)

    # BinaryGrindPeakImageFilter has inverted foreground/background 0 and 1, need to invert
    inverted = sitk.NotImageFilter().Execute(hole_filling)

    largest_component = select_largest_component(inverted)

    contour = sitk.BinaryContourImageFilter().Execute(largest_component)

    plt.imshow(sitk.GetArrayFromImage(contour))
    plt.axis("off")
    plt.show()

    return contour

# Interactive display

In [12]:
interact(
    rotate_and_get_contour_testing,
    img=fixed(image),
    theta_x=(-180, 180),
    theta_y=(-180, 180),
    theta_z=(-180, 180),
    slice_z=(0, image.GetSize()[2] - 1)
)

interactive(children=(IntSlider(value=0, description='theta_x', max=180, min=-180), IntSlider(value=0, descrip…

<function __main__.rotate_and_get_contour_testing(img: SimpleITK.SimpleITK.Image, theta_x: int, theta_y: int, theta_z: int, slice_z: int) -> SimpleITK.SimpleITK.Image>

## Save `sitk` and `np` representations to text files for testing pixel spacing. Also save `sitk` and `np` array representations to PNG files.

# Super weird: How does the `numpy` array look stretched (lost pixel spacing info), whereas the PNG generated from the same array looks fine???

In [13]:
from PIL import Image
# This is the real rotate_and_get_contour function
sitk_contour_slice = rotate_and_get_contour(image, 0, 0, 0, 75)
# show_fiji(contour)

write_sitk_slice(sitk_contour_slice, 'contour_sitk.txt')

# Numpy binary 2D array. It's the transpose of contour_slice
np_contour_slice: np.ndarray = sitk.GetArrayFromImage(sitk_contour_slice)

write_numpy_array_slice_transpose(np_contour_slice, 'contour_numpy.txt')

# Convert binary (0|1) array to binary (0|255) array to display as white in images
np_contour_slice_255: np.ndarray = binary_array_to_255_array(np_contour_slice)

PNG = Image.fromarray(np_contour_slice_255)

PNG.save("numpy_array_255.png")


GradientAnisotropicDiffusionImageFilter (0x104afbe60): Anisotropic diffusion unstable time step: 0.125
Stable time step for this image must be smaller than 0.0997431

GradientAnisotropicDiffusionImageFilter (0x104afbe60): Anisotropic diffusion unstable time step: 0.125
Stable time step for this image must be smaller than 0.0997431

GradientAnisotropicDiffusionImageFilter (0x104afbe60): Anisotropic diffusion unstable time step: 0.125
Stable time step for this image must be smaller than 0.0997431

GradientAnisotropicDiffusionImageFilter (0x104afbe60): Anisotropic diffusion unstable time step: 0.125
Stable time step for this image must be smaller than 0.0997431

GradientAnisotropicDiffusionImageFilter (0x104afbe60): Anisotropic diffusion unstable time step: 0.125
Stable time step for this image must be smaller than 0.0997431



# Get arc length.

`cv2.findContours` must be called on a `numpy` array. `cv2.findContours` returns the same result regardless of whether the array is binary (0|1) or binary (0|255).

See notes on our [Wiki](https://github.com/COMP523TeamD/HeadCircumferenceTool/wiki/Dependencies).

In [23]:
# This returns the same result for binary arrays with 0's and 1's and binary arrays with 0's and 255's, see test_imgproc.py
# contours is a list of contours since there could be multiple contours. That is, contours[0] might be the outer contour and contours[1] might be the inner contour.
# There could be another contour contours[2] (or higher), depending on the image.
contours, hierarchy = cv2.findContours(np_contour_slice, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
contour = contours[0]
circumference_1 = cv2.arcLength(contour, True)

contour = contours[1]
circumference_2 = cv2.arcLength(contour, True)

# contour = contours[2]
# circumference_3 = cv2.arcLength(contour, True)

print(f'I think circumference_1 is the outer contour and circumference 2 is the inner contour but unsure.\nCheck on the NIFTI image with rotation = (0, 0, 0) and slice_z = 150, which has at least 3 contour, which has at least 3 contours.\n')
print(f'Arc length of contour 1: {circumference_1}')
print(f'Arc length of contour 2: {circumference_2}')
# print(f'Circumference 3: {circumference_3}')

[121  77]
I think circumference_1 is the outer contour and circumference 2 is the inner contour but unsure.
Check on the NIFTI image with rotation = (0, 0, 0) and slice_z = 150, which has at least 3 contour, which has at least 3 contours.

Arc length of contour 1: 484.717817902565
Arc length of contour 2: 484.13203144073486
